In [7]:
from txgnn import TxData, TxGNN, TxEval
import pandas as pd
import dgl
import torch
import os
import sys

# Check if CUDA is available
device = 'cpu'
if torch.cuda.is_available():
    print("CUDA is available. Using GPU.")
    device = 'cuda:0'
else:
    print("CUDA is not available. Using CPU.")


CUDA is not available. Using CPU.


In [8]:
%reload_ext autoreload
%autoreload 2

## Loading a Pretrained Model
<p>Saved models available at checkpoints_all_seeds/</p>

In [73]:
# Set device
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

# Initialize data
print("Loading data...")
tx_data = TxData('./data')
tx_data.prepare_split(split='random', seed=42)

# Load model
print("Loading pretrained model...")
tx_gnn = TxGNN(data=tx_data, device=device)
tx_gnn.load_pretrained('./checkpoints_all_seeds/TxGNN_1_random')

# Ensure model is initialized before creating evaluator
if not hasattr(tx_gnn, 'best_model'):
    tx_gnn.best_model = tx_gnn.model

# Initialize evaluator
tx_eval = TxEval(model=tx_gnn)

Using device: cpu
Loading data...
Found local copy...
Found local copy...
Found local copy...
Found saved processed KG... Loading...
Splits detected... Loading splits....
Creating DGL graph....
Done!
Loading pretrained model...


In [75]:
mappings = tx_data.retrieve_id_mapping()

In [121]:
drug_name_to_idx = {}
for idx, id_val in mappings['idx2id_drug'].items():
    try:
        name = mappings['id2name_drug'][id_val]
        drug_name_to_idx[name.lower()] = idx
    except KeyError:
        continue

# Disease name to index mapping
disease_name_to_idx = {}
for idx, id_val in mappings['idx2id_disease'].items():
    try:
        name = mappings['id2name_disease'][id_val]
        disease_name_to_idx[name.lower()] = idx
    except KeyError:
        continue

In [123]:
disease_name_to_idx['burkitt lymphoma']

14708.0

In [127]:
name_to_id['Burkitt lymphoma']

'7243.0'

In [115]:
disease_name = 'burkitt lymphoma'
print(disease_name in disease_name_to_idx)

True


In [80]:
# Get disease index
disease_idx = disease_name_to_idx[disease_name]

# Get all predictions for the disease
print(f"Predicting for disease '{disease_name}'...")
result = tx_eval.eval_disease_centric(
    disease_idxs=[disease_idx],
    relation='indication',
    verbose=False,
    return_raw=True
)

Predicting for disease 'burkitt lymphoma'...


  0%|          | 0/1 [00:00<?, ?it/s]

In [124]:
disease_idx

14708.0

In [ ]:
idx2id_disease

'20181.0'

In [81]:
result

{'prediction': {'7243.0': {'DB00001': -2.1956892,
   'DB00002': -6.318651,
   'DB00003': 0.0,
   'DB00004': -2.4419408,
   'DB00005': 0.5944438,
   'DB00006': 0.080243945,
   'DB00007': 1.7704858,
   'DB00008': -7.3646865,
   'DB00009': -6.654356,
   'DB00010': -4.370191,
   'DB00011': -0.366831,
   'DB00012': -2.086553,
   'DB00013': -1.986274,
   'DB00014': -1.7493756,
   'DB00015': -5.829074,
   'DB00016': 2.2351441,
   'DB00017': -5.9365754,
   'DB00018': -0.3916337,
   'DB00019': -2.3234925,
   'DB00020': -1.1591796,
   'DB00022': -8.625579,
   'DB00023': -0.7231062,
   'DB00024': 0.24938872,
   'DB00025': -9.0131035,
   'DB00026': -1.6268142,
   'DB00027': -3.8444102,
   'DB00028': -1.5353875,
   'DB00029': -1.5354493,
   'DB00030': -9.094618,
   'DB00031': -5.6229115,
   'DB00032': -0.6562753,
   'DB00033': -0.26125807,
   'DB00034': -0.7298143,
   'DB00035': -9.340162,
   'DB00036': -6.2457085,
   'DB00038': -7.320261,
   'DB00039': -1.2243748,
   'DB00040': -4.2023315,
   'DB0

In [ ]:
predictions = result['prediction']['7243.0']

In [89]:
who_filtered = set()
with open('who_essentials/who_filtered_db_ids.txt', 'r') as f:
    for line in f:
        who_filtered.add(line.strip())

print(len(who_filtered))

239


In [93]:
match_scores = {}
for db_id in who_filtered:
    # Check if this drug is in the prediction
    if db_id in predictions:
        match_scores[db_id] = predictions[db_id]

print(match_scores)


{'DB00661': -5.1691456, 'DB01212': -2.2191687, 'DB00593': -0.3047638, 'DB00599': -4.334571, 'DB01147': -4.4657016, 'DB00863': -1.8411293, 'DB09140': -0.3030284, 'DB06715': -6.858951, 'DB00339': -2.993182, 'DB00717': -0.37127364, 'DB00681': 0.16607022, 'DB01118': -3.4449162, 'DB00698': 0.36226022, 'DB01058': -2.9965017, 'DB00335': -5.691082, 'DB00512': -1.2528768, 'DB11121': -1.5618347, 'DB01221': -1.8009813, 'DB00140': -5.748757, 'DB13648': -2.2131126, 'DB01050': -0.6049812, 'DB12864': -1.336663, 'DB00477': -8.858063, 'DB00518': 0.69465566, 'DB00945': -3.5037138, 'DB01168': 4.099097, 'DB00936': 0.7296548, 'DB09278': -0.78664184, 'DB12245': -3.007897, 'DB06694': -14.419096, 'DB00613': -0.9895859, 'DB00653': -5.310964, 'DB00738': -4.403784, 'DB00200': -6.2278943, 'DB00233': -4.023662, 'DB02893': -0.1947447, 'DB00711': -6.8736377, 'DB00795': 1.3136445, 'DB00608': 2.6914077, 'DB01242': 0.22380137, 'DB01096': -3.864702, 'DB00295': -4.9657454, 'DB00291': 4.863766, 'DB01364': -8.64235, 'DB004

In [99]:
# Find a list of drugs that are positive matches
positive_matches = {x: match_scores[x] for x in match_scores if match_scores[x] > 0}


In [103]:
id2name_drug

{'DB09130': 'Copper',
 'DB09140': 'Oxygen',
 'DB00180': 'Flunisolide',
 'DB00240': 'Alclometasone',
 'DB00253': 'Medrysone',
 'DB00324': 'Fluorometholone',
 'DB00394': 'Beclomethasone dipropionate',
 'DB00443': 'Betamethasone',
 'DB00588': 'Fluticasone propionate',
 'DB00591': 'Fluocinolone acetonide',
 'DB00596': 'Ulobetasol',
 'DB00620': 'Triamcinolone',
 'DB00635': 'Prednisone',
 'DB00648': 'Mitotane',
 'DB00663': 'Flumethasone',
 'DB00687': 'Fludrocortisone',
 'DB00741': 'Hydrocortisone',
 'DB00846': 'Flurandrenolide',
 'DB00860': 'Prednisolone',
 'DB00896': 'Rimexolone',
 'DB01013': 'Clobetasol propionate',
 'DB01047': 'Fluocinonide',
 'DB01222': 'Budesonide',
 'DB01384': 'Paramethasone',
 'DB01410': 'Ciclesonide',
 'DB05688': 'CRx-119',
 'DB08906': 'Fluticasone furoate',
 'DB09124': 'Medrogestone',
 'DB13867': 'Fluticasone',
 'DB14538': 'Hydrocortisone aceponate',
 'DB14539': 'Hydrocortisone acetate',
 'DB14540': 'Hydrocortisone butyrate',
 'DB14541': 'Hydrocortisone cypionate',


In [108]:
# Convert to a csv of names and their scores
rows = []
for id, score in positive_matches.items():
    name = id2name_drug[id]
    row = {'drugbank_id': id, 'drug_name': name, 'score': score}
    rows.append(row)
    
df = pd.DataFrame(rows)
os.makedirs('results', exist_ok=True)
df.to_csv('results/matches_indications_full_graph.csv', index=False)
    
    
    